# Extract Simons Observatory instrument parameters

In order to be independent of `sotodlib` and track versions of the instrument model parameters more easily, we extract them to a file included in this repository.

The versioning will be YYYY.MM

In [1]:
import numpy as np

In [2]:
import sotodlib.sim_hardware as hardware

hw = hardware.get_example()

In [3]:
import sotodlib

In [4]:
sotodlib.__version__

'0+untagged.224.g0c64e07'

In [5]:
import h5py

In [6]:
fwhm_arcmin = {"SA" : hw.data["telescopes"]["SAT1"]["fwhm"],
"LA":hw.data["telescopes"]["LAT"]["fwhm"]}

In [7]:
bands = ("LF1", "LF2", "MFF1", "MFF2", "MFS1", "MFS2", "UHF1", "UHF2")
indices = (0, 1, 2, 3, 2, 3, 4, 5)

noise_band_index = dict(zip(bands, indices))

In [8]:
from pathlib import Path

In [9]:
dataset_name = Path("simonsobs_instrument_parameters_2020.06")

In [10]:
!mkdir -p $dataset_name

In [11]:
from collections import OrderedDict
from astropy import units as u
from astropy.table import QTable

In [12]:
channels = []
for tube_id, (tube, tube_properties) in enumerate(hw.data["tubes"].items()):
    bands = [tube_properties["type"] + num for num in "12"]
    telescope = tube[0] + "A"
    for i_band, band in enumerate(bands):
        tag = tube + "_" + band
        print(tag)
        ch = OrderedDict()
        ch["telescope"] = telescope
        ch["tag"] = tag
        ch["band"] = band
        ch["tube"] = tube
        ch["tube_id"] = tube_id
        ch["band_id"] = i_band
        ch["noise_band_index"] = noise_band_index[band]
        ch["fwhm"] = fwhm_arcmin[telescope][band] * u.arcmin
        properties = hw.data["bands"][band]
        ch["center_frequency"] = properties["center"] *u.GHz
        QTable({
            "bandpass_frequency":np.linspace(properties["low"], properties["high"], 10)*u.GHz,
            "bandpass_weight":np.ones(10, dtype=np.float64)
        }).write(dataset_name / f"bandpass_{band}.tbl", format="ascii.ipac", overwrite=True)
        channels.append(ch)

LT0_UHF1
LT0_UHF2
LT1_UHF1
LT1_UHF2
LT2_MFF1
LT2_MFF2
LT3_MFF1
LT3_MFF2
LT4_MFS1
LT4_MFS2
LT5_MFS1
LT5_MFS2
LT6_LF1
LT6_LF2
ST0_UHF1
ST0_UHF2
ST1_MFF1
ST1_MFF2
ST2_MFS1
ST2_MFS2
ST3_LF1
ST3_LF2


In [13]:
instrument_parameters = QTable(channels)

In [14]:
ipac_warning = ["Text file in IPAC table format, read with astropy",
    "from astropy.table import QTable",
    "QTable.read('filename.tbl', format='ascii.ipac')"]

In [15]:

instrument_parameters.meta["comments"] = ["sotodlib_version:" + sotodlib.__version__] + ipac_warning

In [16]:
instrument_parameters

telescope,tag,band,tube,tube_id,band_id,noise_band_index,fwhm,center_frequency
,,,,,,,arcmin,GHz
str2,str8,str4,str3,int64,int64,int64,float64,float64
LA,LT0_UHF1,UHF1,LT0,0,0,4,1.0,225.7
LA,LT0_UHF2,UHF2,LT0,0,1,5,0.9,285.4
LA,LT1_UHF1,UHF1,LT1,1,0,4,1.0,225.7
LA,LT1_UHF2,UHF2,LT1,1,1,5,0.9,285.4
LA,LT2_MFF1,MFF1,LT2,2,0,2,2.2,92.0
LA,LT2_MFF2,MFF2,LT2,2,1,3,1.4,147.5
LA,LT3_MFF1,MFF1,LT3,3,0,2,2.2,92.0
LA,LT3_MFF2,MFF2,LT3,3,1,3,1.4,147.5


In [17]:
instrument_parameters.write(dataset_name / (dataset_name.name+".tbl"), format="ascii.ipac", overwrite=True)